<a href="https://colab.research.google.com/github/wwww6662003/wwww6662003.github.io/blob/master/%E2%80%9CAlpaca_%2B_Llama_3_8b_full_example_ipynb%E2%80%9D%E7%9A%84%E5%BF%AB%E9%80%9F%E4%BD%93%E9%AA%8C%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Llama-3 8b是在疯狂的15万亿令牌上训练的！Llama-2是2万亿。**

要运行这个程序，请在**免费**的Tesla T4 Google Colab实例上按下"*Runtime*"，然后按下"*Run all*"！

要在您自己的计算机上安装Unsloth，请按照unsloth，Github页面上的[安装说明](https://github.com/unslothai/unsloth#installation-instructions---conda)进行操作。

您将学会如何进行[数据准备](#Data)，如何进行[训练](#Train)，如何[运行模型](#Inference)，以及[如何保存它](#Save)（例如Llama.cpp）。

这是我（PM熊叔）在unsloth官方的notebook基础上改的，主要是为了方便大家快速学习如何微调llama，删了不必要的东西，**不建议高手使用**。



## 先环境设置
你按"▶️"就能执行代码

👇来试一下，点击以后提示你添加谷歌云硬盘的授权

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


安装unsloth

In [ ]:
%%capture
import torch

# 由于Colab使用的是torch 2.2.1版本，会导致包损坏，因此必须单独安装
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes


## 下载预训练模型

* 下载要微调的模型，支持16位LoRA或4位QLoRA。
* `max_seq_length`可以设置为任何值，因为我们通过[kaiokendev的](https://kaiokendev.github.io/til)方法自动进行RoPE缩放。
* [**NEW**] 通过[PR 26037](https://github.com/huggingface/transformers/pull/26037)，支持下载4位模型**快4倍**！[我们的仓库](https://huggingface.co/unsloth)有Llama、Mistral 4位模型。


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # 选择任何值！我们在内部自动支持RoPE缩放！
dtype = None # None表示自动检测。对于Tesla T4、V100使用Float16，对于Ampere+使用Bfloat16
load_in_4bit = True # 使用4位量化以减少内存使用。可以为False。

# 我们支持的预4位量化模型，可实现4倍更快的下载速度和无OOM。
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Gemma 7b的Instruct版本
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Gemma 2b的Instruct版本
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15万亿令牌的Llama-3
] # 更多模型请查看https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # 如果使用像meta-llama/Llama-2-7b-hf这样的门控模型，请使用其中一个
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 设置LoRA训练参数
我们现在添加LoRA适配器，所以我们只需要更新所有参数的1到10% !

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # 选择任何大于0的数字！建议使用8、16、32、64、128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # 支持任何值，但等于0时经过优化
    bias = "none",    # 支持任何值，但等于"none"时经过优化
    # [NEW] "unsloth" 使用的VRAM减少30%，适用于2倍更大的批处理大小！
    use_gradient_checkpointing = "unsloth", # True或"unsloth"适用于非常长的上下文
    random_state = 3407,
    use_rslora = False,  # 我们支持排名稳定的LoRA
    loftq_config = None, # 以及LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### 数据准备
我们现在使用来自[yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned)的Alpaca数据集，这是原始[Alpaca数据集](https://crfm.stanford.edu/2023/03/13/alpaca.html)的经过筛选的版本，包含了来自原始数据集中的52000条数据。您可以用您自己的数据准备替换此代码部分。

**[注意]** 要仅训练完成（忽略用户输入），请阅读TRL的文档[此处](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only)。

**[注意]** 记得将**EOS_TOKEN**添加到标记化的输出中！否则将得到无限的生成！

如果您想要在ShareGPT数据集中使用`ChatML`模板，请尝试我们的对话[笔记本](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)。

对于像写小说这样的文本完成任务，请尝试这个[笔记本](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)。


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # 必须添加EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # 必须添加EOS_TOKEN，否则生成将无法停止！
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

<a name="Train"></a>
## 训练模型
现在让我们使用Huggingface TRL的`SFTTrainer`！更多文档请查看：[TRL SFT文档](https://huggingface.co/docs/trl/sft_trainer)。我们执行60步来加快速度，但您可以设置`num_train_epochs=1`进行完整运行，并关闭`max_steps=None`。我们还支持TRL的`DPOTrainer`！

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # 可以使短序列的训练速度提高5倍。
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title 显示当前内存状态
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. 最大内存 = {max_memory} GB.")
print(f"{start_gpu_memory} GB of 内存剩余。")

GPU = Tesla T4. 最大内存 = 14.748 GB.
7.52 GB of 内存剩余。


### ⬇🚀点击这个开始训练！！！！


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.821700
2,2.296200
3,1.692200
4,1.948400
5,1.644700
6,1.604300
7,1.194400
8,1.257400
9,1.106100
10,1.167600


<a name="Inference"></a>
### 测试训练效果
让我们运行模型！您可以更改指令（instruction）和输入（input） - 将输出（output）留空！


In [ ]:
# alpaca_prompt = 从上面复制
FastLanguageModel.for_inference(model) # 启用本机2倍更快的推断
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # 指令 instruction
        "1, 1, 2, 3, 5, 8", # 输入 input
        "", # 输出 output - 生成时请将此留空！
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21, 34, 55, 89, 144<|end_of_text|>']

您还可以使用`TextStreamer`进行连续推断 - 这样您可以逐个标记地查看生成过程，而不必等待全部生成完成！


In [ ]:
# alpaca_prompt = 从上面复制
FastLanguageModel.for_inference(model) # 启用本机2倍更快的推断
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # 指令
        "1, 1, 2, 3, 5, 8", # 输入
        "", # 输出 - 生成时请将此留空！
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144<|end_of_text|>


<a name="Save"></a>
### 保存、加载新模型
要将最终模型保存为 LoRA 适配器，使用 `save_pretrained` 进行本地保存。

**[注意]** 这仅保存 LoRA 适配器，而不是完整模型。要保存为16位或GGUF，请向下滚动！


检测当前地址并模型。

In [ ]:
import os
#获取当前的保存的地址
current_path = os.getcwd()
model.save_pretrained("lora_model") # 本地保存
print(f"保存地址 {current_path}/lora_model")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


保存地址 /content/lora_model


测试LoRA模型：如果您想加载我们刚刚保存的 LoRA 适配器用于推断，请将 `False` 设置为 `True`：

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # 你的训练的模型
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # 指令 instruction
        "", # 输入 input
        "", # 输出 output - 留空!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


KeyboardInterrupt: 

如果您没有安装`unsloth`，您还可以使用Hugging Face的`AutoModelForPeftCausalLM`。但是请注意，这种方法可能会非常慢，因为不支持`4bit`模型下载，并且Unsloth的**推断速度快2倍**。


### GGUF / llama.cpp 转换
现在我们原生支持保存为`GGUF` / `llama.cpp`！我们克隆`llama.cpp`，并默认保存为`q8_0`。我们允许所有方法，如`q4_k_m`。使用`save_pretrained_gguf`进行本地保存，使用`push_to_hub_gguf`上传至HF。

一些支持的量化方法（完整列表请查看我们的[Wiki页面](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)）：

* `f16` 16bit GGUF
* `q4_k_m` - 推荐。对于一半的attention.wv和feed_forward.w2张量使用Q6_K，否则使用Q4_K。



移动合成的模型到谷歌云硬盘，便于下载到本地。

In [ ]:
# Save to q4_k_m GGUF 推荐（完成后将模型放到谷歌云硬盘）
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

# Save to 16bit GGUF（这个体积太大了）
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.65 out of 12.67 RAM for saving.


100%|██████████| 32/32 [02:15<00:00,  4.23s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


NOTICE: llama.cpp GGUF conversion is currently unstable, since llama.cpp is
undergoing some major bug fixes as at 5th of May 2024. This is not an Unsloth issue.
Please be patient - GGUF saving should still work, but might not work as well.
Unsloth: Converting llama model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to q4_k_m will take 20 minutes.
 "-____-"     In total, you will have to wait around 26 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be ./model-unsloth.F16.gguf
This will take 3 minutes...
Unsloth: Conversion completed! Output location: ./model-unsloth.F16.gguf
Unsloth: [2] Converting GGUF 16bit into q4_k_m. This will take 20 minutes...
main: build = 2794 (628b2991)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing './model-unsloth.F16.gguf' to './model-unsloth.Q4_K_M.gguf' as Q4_K_M using 4 threads
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ./model-unsloth.F16

## F16模型量化成Q8，减少模型体积（如果是Q4可以不用）
训练完成model-unsloth.F16.gguf太大了，免费的谷歌云无法保存，量化成Q8可以减少一半的大小

In [ ]:
![ -d "llama.cpp" ] || git clone https://github.com/ggerganov/llama.cpp.git
!cd llama.cpp && make
#!./quantize /content/model-unsloth.F16.gguf /content/model-unsloth.q8_0.gguf q8_0

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_LLAMAFILE  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_LLAMAFILE 
I NVCCFLAGS: -std=c++11 -O3 
I LDFLAGS:    
I CC:        cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
I CXX:       c++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0

make: Nothing to be done for 'default'.

## 将刚刚生成的模型移动到谷歌云
source_path 是你的模型地址
destination_path 是谷歌云地址

In [ ]:
import shutil

source_path = '/content/model-unsloth.Q4_K_M.gguf'
destination_path = '/content/drive/MyDrive/'

# 移动文件，内容有点大需要点时间
shutil.move(source_path, destination_path)
print("请使用谷歌云MyDrive中下载该内容")

'/content/drive/MyDrive/model-unsloth.q8_0.gguf'

下载完毕后，可以至网站：https://lmstudio.ai/ 下载 LM Studio运行